In [1]:
import pymysql
import sklearn
import numpy as np
from SiteClassify import predict_url

In [16]:
db = pymysql.connect(
    host='1.15.220.155',
    user='test',
    password='991125',
    database='spider',
    charset='utf8'
)

In [14]:
c = db.cursor()

c.execute('select distinct(url) from results_test where isPMS is NULL')



results = list(c.fetchall())
count_movie = 0
count_not_movie = 0
count_failed = 0
for i in range(500):
    if (i + 1) % 20 == 0:
        print(' {} number records has been detected. Got {} number movie site, {} not movie site, {} cant request.'.format(i+1, count_movie, count_not_movie, count_failed))
        count_movie = 0
        count_failed = 0
        count_not_movie = 0
    url = results[i][0]
    out = predict_url(url)
    if out == 2:
        count_failed += 1
        m = db.cursor()
        sql = 'update results_test set isPMS=%s where url =%s'
        data = (6, url)
        m.execute(sql, data)
        db.commit()
    if out == 0:
        count_not_movie += 1
        m = db.cursor()
        sql = 'update results_test set isPMS=%s where url =%s'
        data = (7, url)
        m.execute(sql, data)
        db.commit()
    elif out == 1:
        count_movie += 1
        m = db.cursor()
        sql = 'update results_test set isPMS = %s where url = %s'
        data = (8, url)
        m.execute(sql, data)
        db.commit()

 20 number records has been detected. Got 0 number movie site, 11 not movie site, 8 cant request.
 40 number records has been detected. Got 0 number movie site, 14 not movie site, 6 cant request.
 60 number records has been detected. Got 1 number movie site, 12 not movie site, 7 cant request.
 80 number records has been detected. Got 3 number movie site, 12 not movie site, 5 cant request.
 100 number records has been detected. Got 1 number movie site, 15 not movie site, 4 cant request.
 120 number records has been detected. Got 3 number movie site, 16 not movie site, 1 cant request.
 140 number records has been detected. Got 0 number movie site, 10 not movie site, 10 cant request.
 160 number records has been detected. Got 2 number movie site, 13 not movie site, 5 cant request.
 180 number records has been detected. Got 1 number movie site, 10 not movie site, 9 cant request.
 200 number records has been detected. Got 6 number movie site, 14 not movie site, 0 cant request.
 220 number r

ValueError: Unicode strings with encoding declaration are not supported. Please use bytes input or XML fragments without declaration.

In [6]:
from SiteClassify import predict_file
import time
import os
import joblib

model = joblib.load('movie_website.pkl')
tfidf = joblib.load('tf-idf.pkl')

db = pymysql.connect(
    host='1.15.220.155',
    user='test',
    password='991125',
    database='spider',
    charset='utf8'
)


c2 = db.cursor()

c2.execute('select distinct(domain) from unmarked where isPMS_ml is NULL and html = 1')

results = c2.fetchall()

results = [i[0] for i in results]

file_path = '../splashHtmlCrawler/html/unmarked'

dir_results = os.listdir(file_path)

print(len(results))
print(len(dir_results))


crawl_results = []

for i in dir_results:
    if i.split('.html')[0] in results:
        crawl_results.append(i)
        
print('count: {}'.format(len(crawl_results)))

time_a = 0
time_b = time.time()
for index, domain in enumerate(crawl_results, 1):
    if index % 50 == 0:
        time_a = time_b
        time_b = time.time()
        print(' {} number records have been detected, time used: {}'.format(index, time_b - time_a))
    file = os.path.join(file_path, domain)
    out = predict_file(file, model, tfidf)
    m = db.cursor()
    domain = domain.split('.html')[0]
    sql = 'update unmarked set isPMS_ml=%s where domain = %s'
    data = (out, domain)
    m.execute(sql, data)
    db.commit()
    
    
time_b = time.time()
print('time used:{:.2f}s '.format(time_b - time_a))

24
8632
count: 0
time used:1646185759.90s 
